<a href="https://colab.research.google.com/github/loni9164/Text-explainer-issues/blob/main/Bert-shap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Hateful and Offensive Text Classification using pre-trained BERT model**

This notebook demonstrates the use of BERT model 768-layer architecture fine-tuned with a MLP classifier to predict the hateful, offensive and neither texts.

Parts of the model training process have been adapted from the TensorFlow tutorial referred: https://www.tensorflow.org/tutorials/text/classify_text_with_bert



Importing the packages

In [2]:
!pip install icecream
from icecream import ic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
from tensorflow.keras import regularizers, initializers, optimizers, callbacks
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

In [5]:
import seaborn as sb

In [6]:
#train split and fit models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [7]:
import tensorflow as tf
import torch
# specify GPU
device = torch.device("cuda")

In [8]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!pip install sacremoses
from transformers import *
import os
import shutil
import tensorflow_hub as hub
!pip install tensorflow_text
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from transformers import BertTokenizer

Retrieving and reading the Cleaned data

In [11]:
corpus_data = pd.read_csv('https://raw.githubusercontent.com/loni9164/explainable_ai/main/consolidated_ver4.csv')

In [12]:
corpus_data.head()

,narration,transaction_type,class_name,class_id
0,1030970766982876/TOMGOOGLEPLAYMASTERM,Debit,App Payment,0
1,POS 405988XXXXXX2894 GOOGLE *SERVICES POS DEBIT,Debit,App Payment,0
2,POS 405988XXXXXX2894 GOOGLE *SERVICES POS DEBIT,Debit,App Payment,0
3,POS 405988XXXXXX2894 PAYU-WWW.SONYLIV POS DEBIT,Debit,App Payment,0
4,POS 405988XXXXXX2894 GOOGLE *GOOGLE * POS DEBIT,Debit,App Payment,0


In [13]:
# corpus_data.drop(['Unnamed: 0','tweet'],axis=1,inplace=True)

In [14]:
df = corpus_data.copy()
df['combined'] = df['narration'] + '. This Transaction Type is ' + df['transaction_type']

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6166 entries, 0 to 6165
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   narration         6166 non-null   object
 1   transaction_type  6166 non-null   object
 2   class_name        6166 non-null   object
 3   class_id          6166 non-null   int64 
 4   combined          6166 non-null   object
dtypes: int64(1), object(4)
memory usage: 241.0+ KB


In [16]:
df.shape

(6166, 5)

Preprocess the data

In [17]:
# def preprocess_tweet(data):
#     clean = ""
#     tweet = data.split()
#     tweet = [word for word in tweet if len(word) > 1]
#     clean = ' '.join(str(e) for e in tweet)
#     return clean

In [18]:
# df['tweet_clean'] = df['tweet_clean'].apply(preprocess_tweet)

In [19]:
unique_count = np.unique(df['class_name'], return_counts=True)

aa = tuple(zip(*unique_count))

unique_count_pd = pd.DataFrame(aa, columns =['class_name', 'counts'])

op_one = unique_count_pd.loc[unique_count_pd.counts <10]

filtered = unique_count_pd[~(unique_count_pd.counts < 10)]

filter_lst = op_one['class_name'].tolist()

filter_str = '|'.join(filter_lst)

df = df[~df['class_name'].str.contains(filter_str)]

class_op = np.unique(df['class_name'])

print(np.unique(df['class_name'], return_counts=True))

(array(['App Payment', 'Cash Withdrawal', 'Charges', 'Credit Card Payment',
       'Deposit', 'Education', 'Hospital Medical And Diagnostics',
       'Hotel and Restaurant', 'Insurance', 'Interest and Credits',
       'Investment', 'Inward Payment', 'Loan EMI',
       'Movies And Entertainment', 'Others', 'Outward Payment', 'Salary',
       'Shopping', 'Tax Payment', 'Travel and Transport', 'Utility Bill'],
      dtype=object), array([  25,  428,   30,   98,   27,   45,  144,  327,  100,   97,  195,
       1197,   29,   43,  192, 2438,  118,  278,   42,  147,  158]))


In [20]:
df_22 = df.copy()
id = 0
for class_name in class_op:
  df.loc[df['class_name'] == class_name, 'class_id'] = id
  id+=1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [21]:
class_dict = {}
class_reverse = {}
id = 0
for class_name in class_op:
  class_dict[class_name] = id
  class_reverse[id] = class_name
  id+=1

In [22]:
df

,narration,transaction_type,class_name,class_id,combined
0,1030970766982876/TOMGOOGLEPLAYMASTERM,Debit,App Payment,0,1030970766982876/TOMGOOGLEPLAYMASTERM. This Tr...
1,POS 405988XXXXXX2894 GOOGLE *SERVICES POS DEBIT,Debit,App Payment,0,POS 405988XXXXXX2894 GOOGLE *SERVICES POS DEBI...
2,POS 405988XXXXXX2894 GOOGLE *SERVICES POS DEBIT,Debit,App Payment,0,POS 405988XXXXXX2894 GOOGLE *SERVICES POS DEBI...
3,POS 405988XXXXXX2894 PAYU-WWW.SONYLIV POS DEBIT,Debit,App Payment,0,POS 405988XXXXXX2894 PAYU-WWW.SONYLIV POS DEBI...
4,POS 405988XXXXXX2894 GOOGLE *GOOGLE * POS DEBIT,Debit,App Payment,0,POS 405988XXXXXX2894 GOOGLE *GOOGLE * POS DEBI...
...,...,...,...,...,...
6161,VPS/MSW NETRAM /202008200055/023219392918/Mumb...,Debit,Others,14,VPS/MSW NETRAM /202008200055/023219392918/Mumb...
6162,UPI/125214657903/NO REMARKS/9029315708@upi/HDF...,Debit,Outward Payment,15,UPI/125214657903/NO REMARKS/9029315708@upi/HDF...
6163,UPI/208353891228/UPI/rupeshmore16097/HDFC BANK...,Credit,Inward Payment,11,UPI/208353891228/UPI/rupeshmore16097/HDFC BANK...
6164,UPI/131389939450/UPI/ashishanandbt@o/ICICI Ban...,Credit,Inward Payment,11,UPI/131389939450/UPI/ashishanandbt@o/ICICI Ban...


In [23]:
id

21

In [24]:
class_hist = np.bincount(df['class_id'])

total = 0

for class_count in class_hist:
    total += class_count

ic(total)

#{class_id:weight}
weight_cls = {}
cls_id = []
cls_id_nonzero = []


for itr in range(0,len(class_hist)):
    class_id = itr
    cls_id.append(class_id)
    class_count = class_hist[itr]
    if class_count != 0:
        weight_cls[class_id] =  (total / class_count)/(len(class_hist)-1)
        cls_id_nonzero.append(class_id)
    else:
        weight_cls[class_id] =  0

    
ic(weight_cls)
class_weight = weight_cls

ic(len(cls_id_nonzero))
ic(cls_id_nonzero)

ic| total: 6158
ic| weight_cls: {0: 12.315999999999999,
                 1: 0.719392523364486,
                 2: 10.263333333333334,
                 3: 3.141836734693878,
                 4: 11.403703703703703,
                 5: 6.8422222222222215,
                 6: 2.1381944444444443,
                 7: 0.9415902140672783,
                 8: 3.0789999999999997,
                 9: 3.1742268041237116,
                 10: 1.578974358974359,
                 11: 0.25722639933166247,
                 12: 10.617241379310345,
                 13: 7.16046511627907,
                 14: 1.6036458333333332,
                 15: 0.12629204265791633,
                 16: 2.609322033898305,
                 17: 1.1075539568345323,
                 18: 7.330952380952381,
                 19: 2.094557823129252,
                 20: 1.94873417721519}
ic| len(cls_id_nonzero): 21
ic| cls_id_nonzero: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

Splitting the dataset with class ratio maintained

In [25]:
seed = 42
train_text, temp_text, train_labels, temp_labels = train_test_split(df['combined'], df['class_id'], 
                                                                    random_state= seed, 
                                                                    test_size=0.3, 
                                                                    stratify=df['class_id'])

# we will use temp_text and temp_labels to create validation and test set
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state= seed, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

In [26]:
train_text

5575    UPI/128366629629/YouTube Super/googlepay@axisb...
2014       PAYTM NOIDA IN. This Transaction Type is Debit
889     BIL/ONL/000085303882/INDIAN CLE/BSE_VICI988432...
1490    NEFT-N192201184554147-GREAT LAKES E   LEAR-GLS...
2595    IMPS/INET-604615719767-919029315708. This Tran...
                              ...                        
6067    UPI/007320870176/NA/MAYANK DHAWAN/Paytm Paymen...
3239    UPI/P2A/002720627521/GUDDU LAL/Equitas B/UPI. ...
1477    NEFT-N130211499638463-HDFC MUTUAL FUND RED-140...
508     POS 524254XXXXXX0359 LUNCH STOPPER POS DEBIT. ...
4655    VPS/SHAHID BHOS/20130329151451/0              ...
Name: combined, Length: 4310, dtype: object

In [27]:
train_labels.head()

5575    15
2014    15
889     10
1490    11
2595    15
Name: class_id, dtype: int64

In [28]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Prepare to load saved BERT models from TensorFlow hub

To preprocess the text for input to BERT models, TensorFlow Hub provides a matching preprocessing model for each of the BERT models which is also loaded

In [29]:
bert_model_name = 'small_bert/bert_en_uncased_L-12_H-768_A-12' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/2',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [30]:
from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
class_wts = compute_class_weight(class_weight ='balanced',classes = np.unique(train_labels),y = train_labels)

print(class_wts)

[12.07282913  0.68412698  9.77324263  2.97446515 10.80200501  6.62058372
  2.03206035  0.89623622  2.93197279  3.01820728  1.50910364  0.24491419
 10.26190476  6.84126984  1.53162758  0.12030369  2.47274814  1.05250305
  7.0771757   1.99260287  1.84899185]


In [31]:
#Calculation of number of distinct classes
from keras.utils import to_categorical
y_categorical = to_categorical(df['class_id'])
num_classes = y_categorical.shape[1]

Creation of TensorFlow input for train/test/validation set (https://www.tensorflow.org/guide/data)

---



In [32]:
train_text

5575    UPI/128366629629/YouTube Super/googlepay@axisb...
2014       PAYTM NOIDA IN. This Transaction Type is Debit
889     BIL/ONL/000085303882/INDIAN CLE/BSE_VICI988432...
1490    NEFT-N192201184554147-GREAT LAKES E   LEAR-GLS...
2595    IMPS/INET-604615719767-919029315708. This Tran...
                              ...                        
6067    UPI/007320870176/NA/MAYANK DHAWAN/Paytm Paymen...
3239    UPI/P2A/002720627521/GUDDU LAL/Equitas B/UPI. ...
1477    NEFT-N130211499638463-HDFC MUTUAL FUND RED-140...
508     POS 524254XXXXXX0359 LUNCH STOPPER POS DEBIT. ...
4655    VPS/SHAHID BHOS/20130329151451/0              ...
Name: combined, Length: 4310, dtype: object

# Data prepocessing

In [33]:
train_ds = tf.data.Dataset.from_tensor_slices((train_text.values, train_labels.values))
val_ds = tf.data.Dataset.from_tensor_slices((val_text.values, val_labels.values))
test_ds = tf.data.Dataset.from_tensor_slices((test_text.values, test_labels.values))

In [34]:
train_ds = train_ds.shuffle(len(train_text)).batch(32, drop_remainder=False)
train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [35]:
val_ds = val_ds.shuffle(len(val_text)).batch(32, drop_remainder=False)
val_ds

<BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [36]:
test_ds = test_ds.shuffle(len(test_text)).batch(32, drop_remainder=False)
test_ds

<BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [37]:
#print sample tweets in the training set
for feat, targ in train_ds.take(1):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [b'IB BILLPAY DR-HDFCZF-552365XXXXXX7325. This Transaction Type is Debit'
 b'INT CREDIT. This Transaction Type is Credit'
 b'IMPS-706019787656-SOURAV DAS-HDFC-XXXXXX5708-MISCELLANEOUS. This Transaction Type is Credit'
 b'VIN/PAYTM      /201906222046/917315365867/        . This Transaction Type is Debit'
 b'UPI/P2A/006740761086/NITISH  K/State Ban/Payment. This Transaction Type is Credit'
 b'To:9762007700002424:NAVIN DAS. This Transaction Type is Debit'
 b'114501525263:Int.Pd:29-09-2020 to 29-12-2020      . This Transaction Type is Credit'
 b'UPI/824847953255/Rent For Sept 2/8879390725@payt/A. This Transaction Type is Credit'
 b'BIL/001276047751/DTAX/639034020081750             . This Transaction Type is Debit'
 b'NEFT-CMS3342103962329-UPGRAD EDUCATION PRIVATE LIMITED-NEFT-UEDUCATION-CMS33421. This Transaction Type is Credit'
 b'ATM WDL  146749 -CHANDI GHOSH ROAD   KOLKATA   ND  20/05/2021-190728. This Transaction Type is Debit'
 b'IMPS/P2A/006220034683/Nandki/BANKOFI/X011849/

Loading the preprocessing model and BERT model and apply preprocess on text inputs

In [38]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [39]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(1):
    tweet = text_batch.numpy()[i]
    print(f'Tweet: {text_batch.numpy()[i]}')
    label = label_batch.numpy()[i]
    print(f'Label : {label}')

text_test = [tweet]

#sample display of the preprocessed output
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Tweet: b'PAYTM NOIDA IN. This Transaction Type is Debit'
Label : 15
Keys       : ['input_mask', 'input_word_ids', 'input_type_ids']
Shape      : (1, 128)
Word Ids   : [  101  3477 21246  2053  8524  1999  1012  2023 12598  2828  2003  2139]
Input Mask : [1 1 1 1 1 1 1 1 1 1 1 1]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [40]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [41]:
bert_results = bert_model(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1
Pooled Outputs Shape:(1, 768)
Pooled Outputs Values:[-0.2719366  -0.07590407 -0.16654122 -0.0272042   0.25198928 -0.9991556
  0.155921    0.98025167 -0.06393412  0.09493848  0.20306179  0.44379255]
Sequence Outputs Shape:(1, 128, 768)
Sequence Outputs Values:[[-1.68500304e-01 -1.36121005e-01  1.89418852e-01 ...  5.65130830e-01
  -2.30298281e-01 -7.10556924e-01]
 [ 1.03632614e-01  1.57486573e-01  2.39813879e-01 ...  4.91182059e-01
   3.35406512e-04 -7.12035179e-01]
 [-5.94890475e-01  7.51722157e-02  6.69347495e-02 ...  4.80278917e-02
   1.41340852e-01 -6.56024098e-01]
 ...
 [ 1.77791700e-01  2.12074533e-01  9.60354954e-02 ...  9.32759643e-02
  -4.16935146e-01 -4.26381975e-01]
 [-2.01447219e-01  1.31898858e-02  1.64530441e-01 ...  2.32186764e-01
   2.56179005e-01 -5.86950004e-01]
 [-8.91114101e-02  1.32930726e-01 -2.10302606e-01 ...  3.72764856e-01
   1.96857661e-01 -6.68328345e-01]]


Setup of the initial bias (http://karpathy.github.io/2019/04/25/recipe/#2-set-up-the-end-to-end-trainingevaluation-skeleton--get-dumb-baselines)

In [42]:
initial_output_bias = np.array([3.938462, 15, 5.])
initial_output_bias 

array([ 3.938462, 15.      ,  5.      ])

Building the classifier model with fine-tuned MLP

In [43]:
def build_classifier_model(output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dense(512, activation="relu")(net)
    net = tf.keras.layers.Dropout(0.25)(net)
    # net = tf.keras.layers.Dense(3, activation="softmax", name='classifier', bias_initializer=output_bias)(net)
    net = tf.keras.layers.Dense(22, activation="softmax", name='classifier')(net)
    
    return tf.keras.Model(text_input, net)

In [44]:
classifier_model = build_classifier_model(output_bias=initial_output_bias)
bert_raw_result = classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result))

tf.Tensor(
[[0.5114676  0.5064973  0.508454   0.50793743 0.52629256 0.5163199
  0.5207658  0.50525844 0.5109156  0.5278253  0.51143116 0.5108007
  0.5042115  0.51241344 0.5070984  0.5031651  0.5094966  0.50636286
  0.5093327  0.5103279  0.5076241  0.5159081 ]], shape=(1, 22), dtype=float32)


In [45]:
classifier_model.get_weights()[-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.], dtype=float32)

In [46]:
classifier_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

Setup the hyperparameters

In [47]:
# !pip install git+https://github.com/philferriere/cocoapi.git#egg=pycocotools^&subdirectory=PythonAPI

In [48]:
!pip install tf-models-official

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 7.2 MB/s 
     |████████████████████████████████| 636 kB 74.9 MB/s 
     |████████████████████████████████| 1.2 MB 67.5 MB/s 
     |████████████████████████████████| 238 kB 79.6 MB/s 
     |████████████████████████████████| 99 kB 11.0 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 1.1 MB 65.9 MB/s 
     |████████████████████████████████| 92 kB 14.7 MB/s 
     |████████████████████████████████| 352 kB 87.5 MB/s 
     |████████████████████████████████| 48.3 MB 73.5 MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=42be459998a9ceff2a4a5b55c46221ed9b39a7b2eb5b7ba833a2682edbdae057
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none

In [49]:
from tensorflow import keras
from official.nlp import optimization
from keras.callbacks import EarlyStopping

In [50]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

metrics = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')


In [51]:
epochs = 1          #BERT models tend to overfit beyond 5 epochs
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [52]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

Class weight calculation for class imbalance

In [53]:

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)


Training the model with class weights to handle class imbalance

In [54]:
epochs

1

In [55]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_ds,
                               validation_data=val_ds,
                               epochs=epochs,
                               # The class weights go here
                               class_weight=class_weight,
                               callbacks=[es])

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


135/135 [==============================] - 97s 579ms/step - loss: 2.7715 - accuracy: 0.2968 - val_loss: 1.8628 - val_accuracy: 0.7078


In [56]:
result =  classifier_model.predict(test_ds)
print(result.shape)

29/29 [==============================] - 6s 195ms/step
(924, 22)


Converting the test data into dataframe with actual and predicted labels

# Prediction function

In [62]:
def class_predict(texts): 
    if isinstance(texts,list):
      text = tf.data.Dataset.from_tensor_slices((texts))
      return classifier_model.predict(texts)
    elif isinstance(texts,str):
      text = tf.data.Dataset.from_tensor_slices((texts))
      return classifier_model.predict([texts])

In [63]:
predict_fun=class_predict  #model should take the input, preprocess and return the probabilies for each class
X_dataset=['i hate it when i feel fearful for absolutely no reason']
y_dataset=0
class_names=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]     #list
num_features=6   
top_labels=2

# Shap

In [59]:
!pip install shap
import shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 7.3 MB/s 


In [64]:
masker = shap.maskers.Text(r"\W") # this will create a basic whitespace tokenizer
explainer = shap.Explainer(predict_fun, masker, output_names=class_names)
shap_values = explainer(X_dataset)
shap.plots.text(shap_values)

TypeError: ignored